In [105]:
import numpy as np
import datetime as dt
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify

In [106]:
#################################################
# Database Setup
#################################################
#rds_connection_string = "ETLprj:ScottSaid110!@localhost:5432/migration_db"
rds_connection_string = "immigration_cnn:@localhost:5432/migration_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

#'immigration_cnn

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
details = Base.classes.details
regional = Base.classes.regional
regions = Base.classes.regions


In [107]:
Base.classes.keys()

['details', 'regional', 'regions']

In [108]:
pd.read_sql_query('select * from Regional', con=engine).head()

,id,birth_country,age_0_5,age_6_10,age_11_15,age_16_20,age_20_plus,grade_9th,grade_12th,hs_dropout,...,cleaning_and_maint,other_services,sales,administrative,farming_and_fishing,construction,manufacturing,transportation,military,unemployed
0,17,US,0,0,0,0,0,4380278,11086857,587488,...,4183259,8377235,17391363,19613568,916886,7269547,11784060,12504870,556905,1317281
1,18,Mexico,928523,775875,1485420,2238925,5753368,3557555,1818941,26761,...,787212,216708,494685,482108,473083,1316937,1001810,851937,6356,68478
2,19,Eastand Southeast Asia,1500243,946080,926337,941610,4334258,761904,471609,3533,...,146691,388236,525913,559092,15351,87448,465487,297208,10277,39188
3,20,Central Asia,38881,21083,20612,17039,34239,2938,2164,0,...,2776,5757,7961,7078,0,4097,5342,11118,83,509
4,21,South Asia,1148791,589390,446589,470990,1013222,162268,146226,2590,...,27050,62512,273505,157368,3970,18523,92958,149425,1178,24066


In [91]:
demography_json = [

    {demography:"Age",
    labels: ['Age_0-5', "Age_6-10", "Age_11-15", "Age_16-20", "Age_20+"]
    },
   
    {
        US:	[0.000, 0.000, 0.000, 0.000, 0.000],
        Mexico:[0.083, 0.0694, 0.1328, 0.2002, 0.5145],
        Eastand_Southeast_Asia: [0.1735, 0.1094, 0.1071, 0.1089, 0.5012],
        Central_Asia: [0.2949, 0.1599, 0.1563, 0.1292, 0.2597],
        South_Asia: [0.3131, 0.1606, 0.1217, 0.1284, 0.2762],
        Oceania: [0.248, 0.1147, 0.1338, 0.1386, 0.3648 ],
        Europe: [0.1356, 0.074, 0.0855, 0.1308, 0.5741],
        Canada_and_Other_North_America: [0.1833, 0.0816, 0.0761, 0.1053, 0.5537],
        Caribbean: [0.1722, 0.1292, 0.1053, 0.1073, 0.486],
        Central_America: [0.1832, 0.1023, 0.1558, 0.1496, 0.4092],
        South_America: [0.2139, 0.0886, 0.111, 0.1896, 0.3969],
        Middle_EastNorth_Africa: [0.2426, 0.1547, 0.103, 0.1149, 0.3847],
        SubSaharan_Africa: [0.2695, 0.1866, 0.1626, 0.1492, 0.2321],


    }
]

SyntaxError: invalid character in identifier (<ipython-input-91-4a440c685a88>, line 9)

In [210]:
demography = 'age'

labels = pd.read_csv('./CSV/labels.csv')

session = Session(engine)
stmt = session.query(regional).statement
df = pd.read_sql_query(stmt, session.bind)
session.close

meta_dict = {'demography':demography}

if demography == 'age':
    
    demography_df = df.iloc[:,1:7].T
    meta_dict['labels'] =  labels.iloc[0:6,0].to_list()
    
elif demography == 'education':
        demography_df = df.iloc[:,np.r_[1, 7:16]].T
        meta_dict['labels'] =  labels.iloc[6:15,0].to_list()
        
elif demography == 'median_income':
        demography_df = df.iloc[:,np.r_[1, 16:18]].T
        meta_dict['labels'] =  labels.iloc[15:17,0].to_list()

elif demography == 'income':
        demography_df = df.iloc[:,np.r_[1, 18:23]].T
        meta_dict['labels'] =  labels.iloc[17:22,0].to_list()
        
elif demography == 'occupation':
        demography_df = df.iloc[:,np.r_[1, 23:]].T
        meta_dict['labels'] =  labels.iloc[22:,0].to_list()       
    

traces = {demography_df[i].to_list()[0] : demography_df[i].to_list()[1:] for i in demography_df}

myJSON = [meta_dict, traces]
jsonify(myJSON)


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [189]:
list[range(7,16)]

TypeError: 'type' object is not subscriptable